<a href="https://colab.research.google.com/github/jofuture/project/blob/main/AI_02_%EC%A1%B0%EB%AF%B8%EB%9E%98_CS1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np                      # 프레임을 Matrix로 연산하는 라이브러리
import cv2                              # OpenCV 라이브러리
import time                             # 시간측정을 위한 라이브러리
from tkinter import *
from PIL import Image
from PIL import ImageTk
from tkinter import filedialog
import tkinter.scrolledtext as tkst

face_model = './model/res10_300x300_ssd_iter_140000.caffemodel'
face_prototxt = './model/deploy.prototxt.txt' # 모델에 대한 메타 정보
age_model = './model/age_net.caffemodel'
age_prototxt = './model/age_deploy.prototxt'
gender_model = './model/gender_net.caffemodel'
gender_prototxt = './model/gender_deploy.prototxt'

age_list = ['(0-2)', '(4-6)', '(8-12)', '(15-20)', '(25-32)', '(38-43)', '(48-53)', '(60-100)']
gender_list = ['Male','Female']

title_name = 'Age and Gender Recognition'
min_confidence = 0.5 # 최소 신뢰도
recognition_count = 0
elapsed_time = 0
OUTPUT_SIZE = (300, 300)

detector = cv2.dnn.readNetFromCaffe(face_prototxt, face_model)
age_detector = cv2.dnn.readNetFromCaffe(age_prototxt, age_model)
gender_detector = cv2.dnn.readNetFromCaffe(gender_prototxt, gender_model)

    
def detectAndDisplay(image): # 이미지 받고
    start_time = time.time() # 시간 측정
    (h, w) = image.shape[:2] # width , height 가져오기

    # 얼굴 탐지 부분
    imageBlob = cv2.dnn.blobFromImage(image, 1.0, OUTPUT_SIZE, # 1.0은 스케일 값, blob은 사이즈를 바꿔준다
        (104.0, 177.0, 123.0), swapRB=False, crop=False) # min subtract
        # min subtraction      # BGR형태로 바꿀건지
    detector.setInput(imageBlob) # 얼굴 찾기
    detections = detector.forward()

    for i in range(0, detections.shape[2]):
        # 신뢰도 추측
        confidence = detections[0, 0, i, 2]

        # 신뢰도를 높여 약한 탐지를 걸러낸다.
        # 최소 신뢰 이상
        if confidence > min_confidence:
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            # 얼굴의 ROI(region of interest - 관심영역) 를 추출한다.
            face = image[startY:endY, startX:endX]
            (fH, fW) = face.shape[:2]

            # 얼굴의 ROI만으로 BLOB 구성하기
            face_blob = cv2.dnn.blobFromImage(face, 1.0, (227, 227),
                    (78.4263377603, 87.7689143744, 114.895847746),swapRB=False)
            

            # 나이 예측
            # 가장 큰 대응 확률
            age_detector.setInput(face_blob)
            age_predictions = age_detector.forward()
            age_index = age_predictions[0].argmax()
            age = age_list[age_index]
            age_confidence = age_predictions[0][age_index]
            
            gender_detector.setInput(face_blob)
            gender_predictions = gender_detector.forward()
            gender_index = gender_predictions[0].argmax()
            gender = gender_list[gender_index]
            gender_confidence = gender_predictions[0][gender_index]

            text = "{}: {:.2f}% {}: {:.2f}%".format(gender, gender_confidence*100, age, age_confidence*100)
            y = startY - 10 if startY - 10 > 10 else startY + 10
            cv2.rectangle(image, (startX, startY), (endX, endY), # 얼굴 윤곽 그리고
                (0, 255, 0), 2)
            cv2.putText(image, text, (startX, y), # 텍스트 보여주기
                cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 255, 0), 2)
            print('==============================')
            print("Gender {} time {:.2f} %".format(gender, gender_confidence*100))
            print("Age {} time {:.2f} %".format(age, age_confidence*100))
            print("Age     Probability(%)")
            for i in range(len(age_list)):
                print("{}  {:.2f}%".format(age_list[i], age_predictions[0][i]*100))
                
            print("Gender  Probability(%)")
            for i in range(len(gender_list)):
                print("{}  {:.2f} %".format(gender_list[i], gender_predictions[0][i]*100))
                

                
    frame_time = time.time() - start_time
    global elapsed_time
    elapsed_time += frame_time
    print("Frame time {:.3f} seconds".format(frame_time))
    
    cv2.imshow(title_name, image)
    

vs = cv2.VideoCapture(0, cv2.CAP_DSHOW) # 카메라 스트림 연결, 0: 자체 카메라 1: usb카메라,  CAP_DSHOW는 에러방지 
time.sleep(1.0)
if not vs.isOpened: # 카메라가 안열릴 때
    print('### Error opening video ###')
    exit(0)
while True:
    ret, frame = vs.read()
    if frame is None:
        print('### No more frame ###')
        vs.release()
        break
    detectAndDisplay(frame)
    if cv2.waitKey(1) & 0xFF == ord('q'): # 화면 끝낼때 
        break


vs.release() # 메모리 저장 
cv2.destroyAllWindows() # 완전 종료 

